# Michał Matak - Wprowadzenie do sztucznej inteligencji - ćwiczenie 4
## Regresja i klasyfikacja 



### Importowanie bibliotek
* numpy - operacje na macierzach 
* scipy - optymalizacja parametryczna
* matplotlib - tworzenie wykresów

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint
import matplotlib.pyplot as plt

### Funkcje przekształcające strukturę zbioru danych

Pobranie danych z pliku:

In [2]:
def get_data():
    data = []
    with open("iris.data", "r") as file:
        for line in file.readlines():
            splitted_line = line.split(",")
            if len(splitted_line) == 5:
                data.append([list(map(float, splitted_line[:4])), encode(splitted_line[4][:-1])])
        return data
       

Rezultatem jest lista **data** składająca się z list w formie: \[ \[atrybuty\], \[macierz odpowiadająca gatunkowi\] \]

Zakodowanie gatunku kosaćca w macierz:

In [3]:
def encode(name):
    if name == 'Iris-setosa':
        return [1, -1, -1]
    elif name == 'Iris-versicolor':
        return [-1, 1, -1]
    elif name == 'Iris-virginica':
        return [-1, -1, 1]
    else:
        raise Exception("Encoding error")

Podział danych na zbiór testowy według **train_test_ratio**:

In [4]:
def create_train_test_sets(train_test_ratio, data):
    np.random.shuffle(data)
    train_set_size = int(train_test_ratio*len(data))
    test_set_size = len(data) - train_set_size
    train_set = data[:train_set_size]
    test_set = data[-test_set_size:]
    return train_set, test_set

Podział danych w zbiorze ze względu na klasy:

In [5]:
def split_data(data):
    class1 = []
    class2 = []
    class3 = []
    for row in data:
        if row[1][0] == 1:
            class1.append(row)
        elif row[1][1] == 1:
            class2.append(row)
        elif row[1][2] == 1:
            class3.append(row)
        else:
            raise Exception("https://tenor.com/view/thanos-impossible-marvel-shocked-gif-15104180")
    return class1, class2, class3

Tworzenie zbioru danych w formie listy o elementach \[atrybuty, przynależność (1 lub -1) do danej klasy podanej jako argument **index** \].  
Funkcja **make_2D_set** dodatkowo przymuje jako argument indeks kolumn, w których znajdują się wybrane atrybuty

In [6]:
def make_2D_set(feature1, feature2, index, data):
    data_set = np.array([[row[0][feature1], row[0][feature2], row[1][index]] for row in data])
    return data_set


def make_4D_set(index, data):
    data_set = np.array([[row[0][0], row[0][1], row[0][2], row[0][3], row[1][index]] for row in data])
    return data_set

### Funkcje związane z SVM

Funkcja realizująca SVM **linear_SVM** przyjmuje jako argument zbiór z elementami postaci \[atrybuty, przynależność (1 lub -1) do danej klasy\] i zwraca **w** oraz **b** wyznaczające hiperpłaszczyznę dzielącą zbiór.  
Zmiennymi globalnymi są **TRAIN_SET**, **RECORD_SIZE** oraz **LAMBDA**, oznaczające odpowiednio wielkość podanego zbioru, ilość atrybutów dla elementu w zbiorze oraz $\lambda$. Zmienne te są globalne aby nie było potrzeby przekazywania ich jako parametry, do funkcji, która będzie minimalizowana (uznawała by je ona za zmienne i według nich szukała minimum).
Funkcja minimize zaimportowana z biblioteki SciPy wykorzystuje poniżej zdefiniowane funkcje **constraint** oraz **target**.

In [7]:
def linear_SVM(train_set):
    global TRAIN_SET
    global RECORD_SIZE
    global LAMBDA
    LAMBDA = 0.5
    TRAIN_SET = train_set
    RECORD_SIZE = len(TRAIN_SET[0]) - 1
    constraint = NonlinearConstraint(constraint1, np.zeros(len(TRAIN_SET)), np.inf)
    res = minimize(target, np.random.random(RECORD_SIZE + 1), constraints=constraint)
    w = res.x[:RECORD_SIZE]
    b = res.x[RECORD_SIZE]
    return w, b

Funkcja **target** jest równoważna:  

<center>$\sum_i\zeta_i + \lambda \|w\|$</center>
  
      
    
  
która jest minimalizowana dla argumentów *w* i *b* przez **minimize**. Argumenty te są zebrane w jednej liście aby funkcja optymalizująca mogła działać. 

In [8]:
def target(w):
    return np.linalg.norm(w[:RECORD_SIZE], 2)*LAMBDA + dzeta(w).sum()

Na minimalizację powyższej funkcji nałożone są warunki:  
<center>$y_i(w^Tx - b)\geq 1 - \zeta_i$ </center>  
Co jest równoważne:  
<center>$y_i(w^Tx - b) - 1 + \zeta_i\geq 0 $ </center>  
Operacja ta w funkcji constarint jest wykonywana od razu dla całej macierzy, gdzie $\zeta$ jest obliczane od razu jako wektor.

In [9]:
def constraint1(w):
    return ((np.matmul(TRAIN_SET[:, :RECORD_SIZE], w[:RECORD_SIZE]) - w[RECORD_SIZE]) * TRAIN_SET[:, RECORD_SIZE]) + \
            dzeta(w) - np.ones(len(TRAIN_SET))

Funkcja **dzeta** wykonuje operację:
<center>$\zeta_i = max(1 - f(x_i)y_i, 0)$</center>  
dla każdego elementu macierzy z danymi i zwraca wynik jako wektor.


In [10]:
def dzeta(w):
    return np.maximum(np.zeros(len(TRAIN_SET)), np.ones(len(TRAIN_SET)) - ((np.matmul(TRAIN_SET[:, :RECORD_SIZE], w[:RECORD_SIZE]) - w[RECORD_SIZE]) * TRAIN_SET[:, RECORD_SIZE]))

### Uczenie SVM - program

Dane są ładowane do listy data, następnie tworzony jest zbiór testowy, który zostaje podzielony na 3 grupy. Uczone są 3 SVM (porównują zbiory 1 z 2, 2 z 3 i 3 z 1), które zwracają parametry *w* i *b* a ich wyniki są przekazywane do funkcji **predict**.

In [11]:
data = get_data()
train_set, test_set = create_train_test_sets(0.8, data)
set1, set2, set3 = split_data(train_set)

Exception: Encoding error

In [12]:
w_12, b_12 = linear_SVM(make_4D_set(0, set1 + set2))
w_23, b_23 = linear_SVM(make_4D_set(1, set2 + set3))
w_31, b_31 = linear_SVM(make_4D_set(2, set3 + set1))

#### Funkcja predict

Funkcja **predict** przyjmuje jako argument atrybuty elementu zbioru i na ich podstawie decyduje do jakiej klasy należy dany element. Robi to instrukcjami warunkowymi (jeśli widzi, że dla porównań 1 z 2 i 1 z 3 oba wskazania są na 1 to zwraca macierz odpowiadającą elementowi klasy pierwszej, analogicznie dla pozostałych zbiorów). Sytuację, w której porówniane 1 z 2 wskazuje na 2, 2 z 3 na 2 i 3 z 1 na 1 pozostawiłem nierozstrzygnięte. Porównania między dwoma klasami odbywają się za pomocą obliczenia $wx - b$ i zwrócenia 1 jeśli wynik jest większy niż 0 lub -1 jeśli mniejszy.

In [13]:
def predict(x, w_12, w_23, w_31, b_12, b_23, b_31):
    x_lpredict = [threshold_unipolar_function(np.matmul(w_12, x) - b_12, 0),
                  threshold_unipolar_function(np.matmul(w_23, x) - b_23, 0),
                  threshold_unipolar_function(np.matmul(w_31, x) - b_31, 0)]
    if (x_lpredict[2] == -1) and (x_lpredict[0] == 1):
        return [1, -1, -1]
    if (x_lpredict[0] == -1) and (x_lpredict[1] == 1):
        return [-1, 1, -1]
    if (x_lpredict[1] == -1) and (x_lpredict[2] == 1):
        return [-1, -1, 1]

Funkcja progowa unipolarna zwraca 1 jeśli *x* jest większy lub równy *a* i -1 jeśli *x* jest mniejszy od *a*  

In [14]:
def threshold_unipolar_function(x, a):
    if (x >= a):
        return 1
    elif (x < a):
        return -1

### Ewaluacja - sprawdzenie działania algorytmu

Poniżej znajduje się fragment programu, który sprawdza stopień poprawnośći przewidywań przez SVM. Zbiór testowy jest podzielony na 3 klasy i dla każdej klasy przewidywana jest jej klasa na podstawie modelu. Jeśli model się pomylił dodawana jest jedynka do liczby błędów.

In [15]:
tset1, tset2, tset3 = split_data(test_set)
errors1 = 0
errors2 = 0
errors3 = 0

for row in tset1:
    if (predict(row[0], w_12, w_23, w_31, b_12, b_23, b_31) != [1, -1, -1]):
        errors1 += 1
print(f"Class Iris-setosa: accuracy {(1 - errors1/len(tset1))*100:.2f}%")

for row in tset2:
    if (predict(row[0], w_12, w_23, w_31, b_12, b_23, b_31) != [-1, 1, -1]):
        errors2 += 1
print(f"Class Iris-versicolor: accuracy {(1 - errors2/len(tset2))*100:.2f}%")

for row in tset3:
    if (predict(row[0], w_12, w_23, w_31, b_12, b_23, b_31) != [-1, -1, 1]):
        errors3 += 1
print(f"Class Iris-virginica: accuracy {(1 - errors3/len(tset3))*100:.2f}%")
print(f"Total accuracy {(1 - (errors1 + errors2 + errors3)/(len(tset1) + len(tset2) + len(tset3)))*100:.2f}%")

Class Iris-setosa: accuracy 100.00%
Class Iris-versicolor: accuracy 88.89%
Class Iris-virginica: accuracy 100.00%
Total accuracy 96.67%


### Wyniki

Trafność odpowiedzi zależy od wylosowanego zbioru treningowego i zachowania funkcji optymalizującej, ale zwykle przewyższa 90% dla całego zbioru. Przewidywanie przynależności do klasy *Iris_setosa* ma zwykle 100% trafność. 